In [ ]:
# ====== IMPORTS & SETUP ======
from dash import Dash, html, dcc, Input, Output, State, dash_table
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import random

print("Libraries imported successfully")
print("Done ")

Libraries imported successfully


# 2

In [2]:
# ====== DATA LOADING & PROCESSING ======
years = [2019, 2020, 2021, 2022, 2023, 2024]
all_crime_data = {}
crime_categories_data = {}
crime_types_by_year = {}

# Crime type mapping to shorter names
crime_name_mapping = {
    'Vergewaltigung sexuelle Nötigung und sexueller Übergriff im besonders schweren Fall einschl. mit Todesfolge §§ 177 178 StGB': 'Sexualstraftaten',
    'Raub räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252 255 316a StGB': 'Raub & Erpressung',
    'Raub räuberische Erpressung auf/gegen Geldinstitute Postfilialen und -agenturen': 'Raub Banken/Post',
    'Raub räuberische Erpressung auf/gegen sonstige Kassenräume und Geschäfte': 'Raub Geschäfte',
    'Handtaschenraub': 'Handtaschenraub',
    'Sonstige Raubüberfälle auf Straßen Wegen oder Plätzen': 'Raub auf Straßen',
    'Raubüberfälle in Wohnungen': 'Raub in Wohnungen',
    'Gefährliche und schwere Körperverletzung Verstümmelung weiblicher Genitalien §§ 224 226 226a 231 StGB': 'Schwere Körperverletzung',
    'Vorsätzliche einfache Körperverletzung § 223 StGB': 'Einfache Körperverletzung',
    'Widerstand gegen und tätlicher Angriff auf Vollstreckungsbeamte und gleichstehende Personen §§ 113-115 StGB': 'Widerstand gegen Beamte',
    'Widerstand gegen Vollstreckungsbeamte und gleichstehende Personen §§ 113 115 StGB': 'Widerstand gegen Beamte',
    'Tätlicher Angriff auf Vollstreckungsbeamte und gleichstehende Personen §§ 114 115 StGB': 'Angriff auf Beamte',
    'Gewaltkriminalität': 'Gewaltkriminalität',
    'Mord Totschlag und Tötung auf Verlangen': 'Mord & Totschlag'
}

# Load and process data for all years
for year in years:
    try:
        # Main crime data
        df = pd.read_csv(f"PKS/Opfer/{year} Opfer.csv", sep=';', encoding='latin1')
        
        # REMOVE FIRST ROW
        df = df.iloc[1:].reset_index(drop=True)
        
        df.columns.values[2] = 'Stadt'
        df.columns.values[4] = 'Total'
        df['Stadt'] = df['Stadt'].str.strip().str.lower()
        df['Year'] = year
        
        # Apply crime name mapping if Straftat column exists
        if 'Straftat' in df.columns:
            df['Straftat_Short'] = df['Straftat'].map(crime_name_mapping).fillna(df['Straftat'])
        
        # Total crimes per city
        total_crimes = df.groupby('Stadt').agg({'Total':'sum'}).reset_index()
        total_crimes.rename(columns={'Total':'Opfer_insgesamt'}, inplace=True)
        total_crimes['Year'] = year
        all_crime_data[year] = total_crimes  # FIXED: Changed 'total_crimess' to 'total_crimes'
        
        # Store detailed data for crime categories
        crime_categories_data[year] = df
        
        # Get actual crime types from "Straftat" column
        if 'Straftat' in df.columns:
            unique_crimes = df['Straftat'].unique()
            crime_types_by_year[year] = unique_crimes
            print(f"Loaded data for {year}: {len(df)} records, {len(unique_crimes)} crime types (first row removed)")
        else:
            print(f"Loaded data for {year}: {len(df)} records (no Straftat column, first row removed)")
            
    except Exception as e:
        print(f"Error loading data for {year}: {e}")

# Combine all years for trend analysis
all_years_combined = pd.concat([all_crime_data[year] for year in years if year in all_crime_data])

# Get crime categories from actual data (using shortened names)
def get_crime_categories():
    all_crime_types = set()
    
    # Collect crime types from all years
    for year, df in crime_categories_data.items():
        if 'Straftat_Short' in df.columns:
            crime_types = df['Straftat_Short'].dropna().unique()
            all_crime_types.update(crime_types)
        elif 'Straftat' in df.columns:
            # Fallback: use original names and map them
            crime_types = df['Straftat'].dropna().unique()
            short_names = [crime_name_mapping.get(crime, crime) for crime in crime_types]
            all_crime_types.update(short_names)
    
    if all_crime_types:
        # Convert to list and sort for consistency
        crime_list = sorted(list(all_crime_types))
        print(f"Found {len(crime_list)} unique crime types across all years")
        return crime_list
    else:
        # Fallback if no Straftat column
        print("No 'Straftat' column found, using default crime categories")
        return list(crime_name_mapping.values())[:8]

crime_categories = get_crime_categories()

# Print sample crime types for verification
print("\nCrime types with shortened names:")
for i, crime in enumerate(crime_categories):
    print(f"{i+1}. {crime}")

Loaded data for 2019: 18044 records, 15 crime types (first row removed)
Loaded data for 2020: 18044 records, 15 crime types (first row removed)
Loaded data for 2021: 18044 records, 15 crime types (first row removed)
Loaded data for 2022: 18044 records, 16 crime types (first row removed)
Loaded data for 2023: 17999 records, 15 crime types (first row removed)
Loaded data for 2024: 17999 records, 15 crime types (first row removed)
Found 17 unique crime types across all years

Crime types with shortened names:
1. Angriff auf Beamte
2. Einfache Körperverletzung
3. Gefährliche und schwere Körperverletzung Verstümmelung weiblicher Genitalien §§ 224 226 226a 231 StGB 
4. Gewaltkriminalität
5. Handtaschenraub
6. Mord & Totschlag
7. Raub & Erpressung
8. Raub Banken/Post
9. Raub Geschäfte
10. Raub auf Straßen
11. Raub in Wohnungen
12. Raub räuberische Erpressung auf/gegen sonstige Kassenräume und Geschäfte 
13. Raub räuberische Erpressung auf/gegen sonstige Zahlstellen und Geschäfte
14. Schwere K

# 3

In [3]:
# ====== GEO DATA HANDLING ======
try:
    # Load the shapefile
    gdf = gpd.read_file('data/german/shape/geoBoundariesSimplified-3_0_0-DEU-ADM2.shp')
    gdf['shapeName'] = gdf['shapeName'].str.strip().str.lower()
    gdf = gdf.to_crs(epsg=4326)
    print("Geo data loaded successfully")
    
    # Print sample data for debugging
    print(f"Shapefile cities sample: {gdf['shapeName'].head(10).tolist()}")
    if not all_years_combined.empty:
        crime_cities_sample = all_years_combined['Stadt'].unique()[:10]
        print(f"Crime data cities sample: {crime_cities_sample.tolist()}")
    
except Exception as e:
    print(f"Error loading geo data: {e}")
    # Create a simple fallback geo dataframe
    from shapely.geometry import Point
    gdf = gpd.GeoDataFrame({
        'shapeName': ['berlin', 'hamburg', 'munich', 'cologne', 'frankfurt'],
        'geometry': [Point(13.4050, 52.5200), Point(9.9937, 53.5511), Point(11.5820, 48.1351), 
                     Point(6.9603, 50.9375), Point(8.6821, 50.1109)]
    })
    print("Using fallback geo data")

Geo data loaded successfully
Shapefile cities sample: ['kreis schleswig-flensburg', 'kreis steinburg', 'kreis rendsburg-eckernf', 'kreis nordfriesland', 'flensburg', 'kiel', 'kreis ostholstein', 'kreis plã¶n', 'lã¼beck', 'kreis dithmarschen']
Crime data cities sample: ['aachen', 'ahrweiler', 'aichach-friedberg', 'alb-donau-kreis', 'altenburger land', 'altenkirchen (westerwald)', 'altmarkkreis salzwedel', 'altötting', 'alzey-worms', 'amberg']


# 4

In [4]:
# ====== APP INITIALIZATION & CSS STYLING ======
app = Dash(__name__, suppress_callback_exceptions=True)
app.title = "Crime Data Visualization: Germany"

# ====== RESPONSIVE STYLING ======
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .kpi-card {
                background: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                text-align: center;
                border-left: 4px solid #007bff;
                transition: transform 0.2s ease;
                min-height: 100px;
                display: flex;
                flex-direction: column;
                justify-content: center;
            }
            .kpi-card:hover {
                transform: translateY(-2px);
                box-shadow: 0 4px 15px rgba(0,0,0,0.15);
            }
            .chart-container {
                background: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                width: 100%;
            }
            .chart-container-full {
                background: white;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                width: 100%;
                grid-column: 1 / -1;
            }
            body {
                margin: 0;
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                background-color: #f8f9fa;
            }
            .dark-mode {
                background-color: #1a1a1a;
                color: #ffffff;
            }
            .dark-mode .kpi-card {
                background: #2d3748;
                color: #ffffff;
            }
            .dark-mode .chart-container {
                background: #2d3748;
                color: #ffffff;
            }
            .dark-mode .chart-container-full {
                background: #2d3748;
                color: #ffffff;
            }
            .dark-mode #sidebar {
                background-color: #2d3748;
                color: #ffffff;
            }
            .dark-mode #main-content {
                background-color: #1a1a1a;
            }
            
            /* Dropdown styling for dark mode */
            .dark-mode .Select-control {
                background-color: #4a5568 !important;
                border-color: #718096 !important;
                color: #ffffff !important;
            }
            .dark-mode .Select-value-label {
                color: #ffffff !important;
            }
            .dark-mode .Select-menu-outer {
                background-color: #4a5568 !important;
                border-color: #718096 !important;
                color: #ffffff !important;
            }
            .dark-mode .Select-option {
                background-color: #4a5568 !important;
                color: #ffffff !important;
            }
            .dark-mode .Select-option.is-focused {
                background-color: #718096 !important;
                color: #ffffff !important;
            }
            .dark-mode .Select-input input {
                color: #ffffff !important;
            }
            .dark-mode .Select-placeholder {
                color: #a0aec0 !important;
            }
            .dark-mode .Select--single > .Select-control .Select-value {
                color: #ffffff !important;
            }
            
            /* Additional dark mode styling for form elements */
            .dark-mode .rc-slider-track {
                background-color: #4299e1 !important;
            }
            .dark-mode .rc-slider-handle {
                border-color: #4299e1 !important;
                background-color: #2d3748 !important;
            }
            .dark-mode .rc-slider-rail {
                background-color: #4a5568 !important;
            }
            
            /* Table styling for dark mode */
            .dark-mode .dash-table-container .dash-spreadsheet-container .dash-spreadsheet-inner table {
                background-color: #2d3748 !important;
                color: #ffffff !important;
            }
            .dark-mode .dash-table-container .dash-spreadsheet-container .dash-spreadsheet-inner th {
                background-color: #4a5568 !important;
                color: #ffffff !important;
                border-color: #718096 !important;
            }
            .dark-mode .dash-table-container .dash-spreadsheet-container .dash-spreadsheet-inner td {
                background-color: #2d3748 !important;
                color: #ffffff !important;
                border-color: #718096 !important;
            }
            
            /* Mobile Responsive */
            @media (max-width: 768px) {
                #sidebar {
                    width: 280px !important;
                }
                #sidebar[style*="left: -400px"] {
                    left: -400px !important;
                }
                #sidebar[style*="left: 20px"] {
                    left: 20px !important;
                }
                #main-content {
                    margin-left: 20px !important;
                    padding: 15px !important;
                }
                .kpi-card {
                    padding: 15px;
                    min-height: 80px;
                }
                .chart-container {
                    padding: 15px;
                }
                .chart-container-full {
                    padding: 15px;
                }
            }
            
            @media (max-width: 480px) {
                #main-content {
                    margin-left: 10px !important;
                    padding: 10px !important;
                }
                .kpi-card {
                    padding: 10px;
                    min-height: 70px;
                }
                .chart-container {
                    padding: 10px;
                }
                .chart-container-full {
                    padding: 10px;
                }
            }
        </style>
        <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

print("App initialized and CSS styling applied successfully")

App initialized and CSS styling applied successfully


# 5

In [5]:
# ====== DASHBOARD LAYOUT ======
app.layout = html.Div([
    dcc.Store(id="theme-store", data="light"),
    dcc.Store(id="sidebar-store", data=True),
    dcc.Store(id="selected-city", data=""),
    dcc.Store(id="selected-crime-type", data=crime_categories[0] if crime_categories else ""),
    
    # Fixed Header - Responsive
    html.Div([
        html.Button("☰", id="toggle-sidebar", n_clicks=0,
                   style={"padding": "10px", "borderRadius": "5px", 
                          "border": "1px solid #ddd", "backgroundColor": "white", 
                          "cursor": "pointer", "marginRight": "15px"}),
        html.H1("Crime Data Visualization: Germany 🇩🇪", 
                style={"margin": "0", "color": "#2c3e50", "fontSize": "clamp(18px, 4vw, 24px)", "textAlign": "center", "flex": "1"}),
        html.Button("🌙", id="theme-toggle", n_clicks=0,
                   style={"marginLeft": "auto", "padding": "10px", "borderRadius": "5px",
                          "border": "1px solid #ddd", "backgroundColor": "white", "cursor": "pointer"})
    ], style={
        "position": "fixed",
        "top": "0",
        "left": "0",
        "right": "0",
        "display": "flex",
        "alignItems": "center",
        "padding": "15px 20px",
        "backgroundColor": "#f8f9fa",
        "borderBottom": "2px solid #dee2e6",
        "zIndex": "1000",
        "boxShadow": "0 2px 10px rgba(0,0,0,0.1)"
    }),
    
    # Main Content Area
    html.Div([
        # Left Sidebar - Responsive
        html.Div([
            html.H3("Filters & Controls", style={"marginBottom": "20px", "color": "#2c3e50", "textAlign": "center", "fontSize": "clamp(16px, 3vw, 20px)"}),
            
            html.Div([
                html.Label("Select Year Range:", style={"fontWeight": "bold", "marginBottom": "10px", "fontSize": "14px"}),
                dcc.RangeSlider(
                    id='year-slider',
                    min=min(years),
                    max=max(years),
                    value=[min(years), max(years)],
                    marks={str(year): str(year) for year in years},
                    step=1
                ),
            ], style={"marginBottom": "25px"}),
            
            html.Div([
                html.Label("Select Crime Type:", style={"fontWeight": "bold", "marginBottom": "10px", "fontSize": "14px"}),
                dcc.Dropdown(
                    id='crime-type-dropdown',
                    options=[{'label': crime, 'value': crime} for crime in crime_categories],
                    value=crime_categories[0] if crime_categories else "",
                    clearable=False,
                ),
            ], style={"marginBottom": "25px"}),
            
            html.Div([
                html.Label("Compare Cities:", style={"fontWeight": "bold", "marginBottom": "10px", "fontSize": "14px"}),
                dcc.Dropdown(
                    id='top-cities-dropdown',
                    options=[],
                    multi=True,
                    placeholder="Select cities to compare...",
                ),
            ], style={"marginBottom": "25px"}),
            
            html.Div([
                html.Label("Select Cities for Crime Type Trends:", style={"fontWeight": "bold", "marginBottom": "10px", "fontSize": "14px"}),
                dcc.Dropdown(
                    id='crime-trend-cities-dropdown',
                    options=[],
                    multi=True,
                    value=['berlin', 'hamburg', 'munich', 'cologne', 'frankfurt'],
                    placeholder="Select cities for crime type trends...",
                ),
            ], style={"marginBottom": "25px"}),
            
            html.Hr(),
            html.H4("Quick Stats", style={"color": "#2c3e50", "marginBottom": "15px", "fontSize": "16px"}),
            html.Div(id="quick-stats", style={"fontSize": "14px", "lineHeight": "1.6"})
            
        ], id="sidebar", style={
            "width": "clamp(280px, 25vw, 320px)",
            "backgroundColor": "white", 
            "padding": "20px", 
            "borderRadius": "10px",
            "boxShadow": "0 2px 10px rgba(0,0,0,0.1)",
            "height": "calc(100vh - 100px)",
            "position": "fixed",
            "top": "80px",
            "left": "20px",
            "overflowY": "auto",
            "transition": "all 0.3s ease",
            "zIndex": "999"
        }),
        
        # Main Content - Responsive
        html.Div([
            # KPI Cards Row - Responsive
            html.Div([
                html.Div([
                    html.H4("Total Crimes", style={"margin": "0 0 10px 0", "fontSize": "clamp(12px, 2vw, 14px)", "color": "#6c757d"}),
                    html.H2(id="total-crimes-kpi", style={"margin": "0", "color": "#dc3545", "fontSize": "clamp(20px, 4vw, 28px)"})
                ], className="kpi-card"),
                
                html.Div([
                    html.H4("Average per City", style={"margin": "0 0 10px 0", "fontSize": "clamp(12px, 2vw, 14px)", "color": "#6c757d"}),
                    html.H2(id="avg-crimes-kpi", style={"margin": "0", "color": "#fd7e14", "fontSize": "clamp(20px, 4vw, 28px)"})
                ], className="kpi-card"),
                
                html.Div([
                    html.H4("Highest Crime City", style={"margin": "0 0 10px 0", "fontSize": "clamp(12px, 2vw, 14px)", "color": "#6c757d"}),
                    html.H2(id="highest-crime-kpi", style={"margin": "0", "color": "#6f42c1", "fontSize": "clamp(16px, 3vw, 28px)"})
                ], className="kpi-card"),
                html.Div([
                    html.H4("Lowest Crime City", style={"margin": "0 0 10px 0", "fontSize": "clamp(12px, 2vw, 14px)", "color": "#6c757d"}),
                    html.H2(id="lowest-crime-kpi", style={"margin": "0", "color": "#20c997", "fontSize": "clamp(16px, 3vw, 28px)"})
                    ], className="kpi-card"),
               
            ], style={
                "display": "grid", 
                "gridTemplateColumns": "repeat(auto-fit, minmax(200px, 1fr))",
                "gap": "15px", 
                "marginBottom": "25px"
            }),
            
            # LARGE MAP SECTION - TWICE AS BIG
            html.Div([
                html.Div([
                    html.H4("Crime Distribution Map", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-map", style={"height": "80vh", "borderRadius": "10px"})  # Increased height
                ], className="chart-container-full"),  # Changed to full width
            ], style={
                "marginBottom": "25px"
            }),
            
            # Map and Trends Row - Responsive (now just the top cities chart)
            html.Div([
                html.Div([
                    html.H4("Top 10 Cities by Crime Rate", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="top-cities-chart", style={"height": "50vh", "borderRadius": "10px"})  # Regular height
                ], className="chart-container"),
                
                html.Div([
                    html.H4("Crime Trends Over Years", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-trends-chart", style={"height": "50vh", "borderRadius": "10px"})  # Regular height
                ], className="chart-container"),
            ], style={
                "display": "grid", 
                "gridTemplateColumns": "repeat(auto-fit, minmax(400px, 1fr))",
                "gap": "20px", 
                "marginBottom": "25px"
            }),
            
            # NEW SECTION: Crime Type Trends by City
            html.Div([
                html.Div([
                    html.H4("Crime Type Trends by City (2019-2024)", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-type-trends-chart", style={"height": "60vh", "borderRadius": "10px"})
                ], className="chart-container-full"),
            ], style={
                "marginBottom": "25px"
            }),
            
            # Crime Patterns Row - Responsive
            html.Div([
                html.Div([
                    html.H4("Specific Crime Type Analysis", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-type-analysis", style={"height": "50vh", "borderRadius": "10px"})
                ], className="chart-container"),
                
                html.Div([
                    html.H4("Seasonal Crime Patterns", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="seasonal-patterns-chart", style={"height": "50vh", "borderRadius": "10px"})
                ], className="chart-container"),
            ], style={
                "display": "grid", 
                "gridTemplateColumns": "repeat(auto-fit, minmax(350px, 1fr))",
                "gap": "20px", 
                "marginBottom": "25px"
            }),
            
            # Crime Type Distribution Row (REMOVED CRIME HOTSPOTS)
            html.Div([
                html.Div([
                    html.H4("Crime Type Distribution", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-type-distribution", style={"height": "50vh", "borderRadius": "10px"})
                ], className="chart-container"),
                
                html.Div([
                    html.H4("Crime Rate Analysis", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-density-correlation", style={"height": "50vh", "borderRadius": "10px"})
                ], className="chart-container"),
            ], style={
                "display": "grid", 
                "gridTemplateColumns": "repeat(auto-fit, minmax(350px, 1fr))",
                "gap": "20px", 
                "marginBottom": "25px"
            }),
            
            # Advanced Analytics Row - Responsive
            html.Div([
                html.Div([
                    html.H4("Crime Forecast (Next Year)", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                    dcc.Graph(id="crime-forecast", style={"height": "50vh", "borderRadius": "10px"})
                ], className="chart-container-full"),
            ], style={
                "marginBottom": "25px"
            }),
            
            # Data Table - Responsive
            html.Div([
                html.H4("Detailed Crime Data", style={"marginBottom": "15px", "color": "#2c3e50", "fontSize": "clamp(16px, 3vw, 18px)"}),
                dash_table.DataTable(
                    id='crime-data-table',
                    columns=[],
                    data=[],
                    page_size=10,
                    style_table={
                        'overflowX': 'auto', 
                        'borderRadius': '10px',
                        'minWidth': '100%'
                    },
                    style_cell={
                        'textAlign': 'left',
                        'padding': '12px',
                        'whiteSpace': 'normal',
                        'height': 'auto',
                        'border': '1px solid #dee2e6',
                        'minWidth': '80px',
                        'fontSize': '14px'
                    },
                    style_header={
                        'backgroundColor': '#f8f9fa',
                        'fontWeight': 'bold',
                        'border': '1px solid #dee2e6',
                        'fontSize': '14px'
                    }
                )
            ], className="chart-container-full")
            
        ], id="main-content", style={
            "marginLeft": "clamp(320px, 27vw, 360px)",
            "padding": "20px",
            "minHeight": "100vh",
            "backgroundColor": "#f8f9fa",
            "maxWidth": "100%",
            "transition": "all 0.3s ease",
            "overflowX": "hidden"
        })
    ], style={
        "marginTop": "80px",
        "position": "relative"
    })
])

print("Dashboard layout created successfully")

Dashboard layout created successfully


# 6

In [6]:
# ====== CALLBACKS - THEME & SIDEBAR ======

# Theme toggle callback
@app.callback(
    [Output("theme-store", "data"),
     Output("theme-toggle", "children"),
     Output("main-content", "className")],
    [Input("theme-toggle", "n_clicks")],
    [State("theme-store", "data")]
)
def toggle_theme(n_clicks, current_theme):
    if n_clicks is None or n_clicks == 0:
        return current_theme, "🌙", ""
    
    new_theme = "dark" if current_theme == "light" else "light"
    icon = "☀️" if new_theme == "dark" else "🌙"
    theme_class = "dark-mode" if new_theme == "dark" else ""
    
    return new_theme, icon, theme_class

# Fixed Sidebar toggle callback
@app.callback(
    [Output("sidebar-store", "data"),
     Output("sidebar", "style"),
     Output("main-content", "style"),
     Output("toggle-sidebar", "children")],
    [Input("toggle-sidebar", "n_clicks"),
     Input("theme-store", "data")],
    [State("sidebar-store", "data")]
)
def toggle_sidebar(n_clicks, theme, sidebar_open):
    if n_clicks is None:
        n_clicks = 0
    
    # Toggle sidebar state when button is clicked
    if n_clicks > 0:
        sidebar_open = not sidebar_open
    
    if sidebar_open:
        # Sidebar open
        sidebar_style = {
            "width": "clamp(280px, 25vw, 320px)",
            "left": "20px",
            "backgroundColor": "white", 
            "padding": "20px", 
            "borderRadius": "10px",
            "boxShadow": "0 2px 10px rgba(0,0,0,0.1)",
            "height": "calc(100vh - 100px)",
            "position": "fixed",
            "top": "80px",
            "overflowY": "auto",
            "transition": "all 0.3s ease",
            "zIndex": "999"
        }
        content_style = {
            "marginLeft": "clamp(320px, 27vw, 360px)",
            "padding": "20px",
            "minHeight": "100vh",
            "backgroundColor": "#f8f9fa",
            "maxWidth": "100%",
            "transition": "all 0.3s ease",
            "overflowX": "hidden"
        }
        button_icon = "✕"
    else:
        # Sidebar closed
        sidebar_style = {
            "width": "clamp(280px, 25vw, 320px)",
            "left": "-400px",  # Move completely off-screen
            "backgroundColor": "white", 
            "padding": "20px", 
            "borderRadius": "10px",
            "boxShadow": "0 2px 10px rgba(0,0,0,0.1)",
            "height": "calc(100vh - 100px)",
            "position": "fixed",
            "top": "80px",
            "overflowY": "auto",
            "transition": "all 0.3s ease",
            "zIndex": "999"
        }
        content_style = {
            "marginLeft": "20px",
            "padding": "20px",
            "minHeight": "100vh",
            "backgroundColor": "#f8f9fa",
            "maxWidth": "100%",
            "transition": "all 0.3s ease",
            "overflowX": "hidden"
        }
        button_icon = "☰"
    
    # Apply theme colors
    if theme == "dark":
        sidebar_style['backgroundColor'] = '#2d3748'
        sidebar_style['color'] = 'white'
        content_style['backgroundColor'] = '#1a1a1a'
    else:
        sidebar_style['backgroundColor'] = 'white'
        content_style['backgroundColor'] = '#f8f9fa'
    
    return sidebar_open, sidebar_style, content_style, button_icon

print("Theme and sidebar callbacks defined")

Theme and sidebar callbacks defined


# 7

In [7]:
# ====== CALLBACKS - DATA UPDATES ======

# Update top cities dropdown
@app.callback(
    [Output("top-cities-dropdown", "options"),
     Output("crime-trend-cities-dropdown", "options")],
    [Input("year-slider", "value")]
)
def update_cities_dropdown(year_range):
    start_year, end_year = year_range
    filtered_data = all_years_combined[
        (all_years_combined['Year'] >= start_year) & 
        (all_years_combined['Year'] <= end_year)
    ]
    
    if filtered_data.empty:
        return [], []
    # getting the cities with highest crime rate
    top_cities = filtered_data.groupby('Stadt')['Opfer_insgesamt'].sum().nlargest(20)
    cities_options = [{'label': city.title(), 'value': city} for city in top_cities.index]
    
    # For crime trend cities, include major German cities
    major_cities = ['berlin', 'hamburg', 'munich', 'cologne', 'frankfurt', 'stuttgart', 
                   'düsseldorf', 'dortmund', 'essen', 'leipzig', 'bremen', 'dresden',
                   'hannover', 'nürnberg', 'duisburg', 'bochum', 'wuppertal', 'bielefeld']
    
    available_major_cities = [city for city in major_cities if city in top_cities.index]
    crime_trend_options = [{'label': city.title(), 'value': city} for city in available_major_cities]
    
    return cities_options, crime_trend_options

# Update KPI cards and quick stats
# Update KPI cards and quick stats
# Update KPI cards and quick stats
@app.callback(
    [Output("total-crimes-kpi", "children"),
     Output("avg-crimes-kpi", "children"),
     Output("highest-crime-kpi", "children"),
     Output("lowest-crime-kpi", "children"),  # CHANGED: from yoy-change-kpi to lowest-crime-kpi
     Output("quick-stats", "children")],
    [Input("year-slider", "value")]
)
def update_kpis(year_range):
    start_year, end_year = year_range
    filtered_data = all_years_combined[
        (all_years_combined['Year'] >= start_year) & 
        (all_years_combined['Year'] <= end_year)
    ]
    
    if filtered_data.empty:
        return "0", "0", "N/A", "N/A", "No data available for selected range"
    
    total_crimes = filtered_data['Opfer_insgesamt'].sum()
    avg_crimes = filtered_data.groupby('Stadt')['Opfer_insgesamt'].sum().mean()
    city_crimes = filtered_data.groupby('Stadt')['Opfer_insgesamt'].sum()
    
    # Highest crime city
    if not city_crimes.empty:
        highest_city = city_crimes.idxmax()
        highest_value = city_crimes.max()
    else:
        highest_city = "N/A"
        highest_value = 0
    
    # NEW: Lowest crime city
    if not city_crimes.empty:
        lowest_city = city_crimes.idxmin()
        lowest_value = city_crimes.min()
    else:
        lowest_city = "N/A"
        lowest_value = 0
    
    # Quick stats
    total_cities = filtered_data['Stadt'].nunique()
    avg_per_city = f"{avg_crimes:,.0f}"
    
    quick_stats = html.Div([
        html.P(f"📊 Total Cities: {total_cities}"),
        html.P(f"📈 Average per City: {avg_per_city}"),
        html.P(f"🔥 Highest: {highest_city.title() if highest_city != 'N/A' else 'N/A'}"),
        html.P(f"🟢 Lowest: {lowest_city.title() if lowest_city != 'N/A' else 'N/A'}"),
        html.P(f"📅 Period: {start_year}-{end_year}")
    ])
    
    return (
        f"{total_crimes:,.0f}",
        f"{avg_crimes:,.0f}",
        highest_city.title() if highest_city != "N/A" else "N/A",
        lowest_city.title() if lowest_city != "N/A" else "N/A",  # NEW: Return lowest crime city
        quick_stats
    )

print("Data update callbacks defined")

Data update callbacks defined


# 8

In [8]:
# ====== MAP CREATION FUNCTION ======
def create_crime_map(data, template, year_range):
    try:
        if data.empty:
            fig = go.Figure()
            fig.update_layout(
                title="No crime data available for selected range",
                template=template,
                height=500
            )
            return fig
        
        # Aggregate crime data by city
        crime_by_city = data.groupby('Stadt')['Opfer_insgesamt'].sum().reset_index()
        
        # Merge with geo data using left join to keep all geographical areas
        merged = gdf.merge(crime_by_city, left_on='shapeName', right_on='Stadt', how='left')
        merged['Opfer_insgesamt'] = merged['Opfer_insgesamt'].fillna(0)  # Fill NaN with 0
        
        print(f"Map data for {year_range} - Cities with crime data: {len(merged[merged['Opfer_insgesamt'] > 0])}")
        print(f"Map data for {year_range} - Total areas: {len(merged)}")
        
        # Convert to GeoJSON
        geojson_data = json.loads(merged.to_json())
        
        # Create the map using choropleth_mapbox
        fig = px.choropleth_mapbox(
            merged,
            geojson=geojson_data,
            locations='shapeName',
            featureidkey='properties.shapeName',
            color='Opfer_insgesamt',
            color_continuous_scale=["#ffe6e6", "#ff9999", "#ff4d4d", "#ff0000"],
            mapbox_style="carto-positron",
            center={"lat": 51.1657, "lon": 10.4515},
            zoom=5.5,  # Increased zoom level (default was 4)
            opacity=0.7,
            hover_name='shapeName',
            hover_data={'Opfer_insgesamt': True, 'shapeName': False},
            title=f"Total Crime Victims by City/District ({year_range[0]}-{year_range[1]})"
        )
        
        # Try to add city labels if we have centroid data
        try:
            centroids = merged.geometry.centroid
            fig.add_scattermapbox(
                lat=centroids.y,
                lon=centroids.x,
                mode='text',
                text=merged['shapeName'].str.title(),
                textposition="top center",
                showlegend=False,
                hoverinfo='skip',
                textfont=dict(size=9, color="black" if template=="plotly_white" else "white")
            )
        except Exception as e:
            print(f"Could not add city labels: {e}")
        
        fig.update_layout(
            margin={"r": 0, "t": 40, "l": 0, "b": 0},
            template=template
        )
        
        return fig
        
    except Exception as e:
        print(f"Error creating map: {e}")
        # Fallback: create a simple map with scatter points
        fig = go.Figure()
        
        # Add scatter points for cities with crime data
        try:
            centroids = merged.geometry.centroid
            fig.add_trace(go.Scattermapbox(
                lat=centroids.y,
                lon=centroids.x,
                mode='markers+text',
                text=merged['shapeName'].str.title(),
                textposition="top center",
                marker=dict(
                    size=8,
                    color=merged['Opfer_insgesamt'],
                    colorscale="reds",
                    showscale=True,
                    colorbar=dict(title="Crime Victims")
                ),
                hovertemplate=(
                    "<b>%{text}</b><br>" +
                    "Victims: %{marker.color}<br>" +
                    "<extra></extra>"
                )
            ))
        except:
            # Ultimate fallback: just show Germany outline
            fig.add_trace(go.Scattermapbox(
                lat=[51.1657],
                lon=[10.4515],
                mode='text',
                text=["Germany"],
                textposition="middle center",
                textfont=dict(size=20, color="red")
            ))
        
        fig.update_layout(
            mapbox=dict(
                style="carto-positron",
                center=dict(lat=51.1657, lon=10.4515),
                zoom=5.5  # Increased zoom level for fallback too
            ),
            margin={"r": 0, "t": 40, "l": 0, "b": 0},
            title=f"Crime Distribution Across Germany ({year_range[0]}-{year_range[1]})",
            template=template
        )
        
        return fig

print("Map creation function defined")

Map creation function defined


# 9

In [9]:
# ====== CHART CREATION FUNCTIONS ======
def create_top_cities_chart(data, template):
    if data.empty:
        fig = go.Figure()
        fig.update_layout(title="No data available", template=template, height=500)
        return fig
    
    top_cities = data.groupby('Stadt')['Opfer_insgesamt'].sum().nlargest(10)
    
    fig = px.bar(
        x=top_cities.values,
        y=[city.title() for city in top_cities.index],
        orientation='h',
        title="Top 10 Cities by Total Crimes",
        labels={'x': 'Total Crimes', 'y': 'City'},
        color=top_cities.values,
        color_continuous_scale='reds'
    )
    
    fig.update_layout(template=template, height=500, showlegend=False)
    return fig

def create_trends_chart(start_year, end_year, selected_cities, template):
    if selected_cities:
        trend_data = all_years_combined[all_years_combined['Stadt'].isin(selected_cities)]
        if trend_data.empty:
            fig = go.Figure()
            fig.update_layout(title="No data for selected cities", template=template, height=400)
            return fig
            
        fig = px.line(
            trend_data,
            x='Year',
            y='Opfer_insgesamt',
            color='Stadt',
            title="Crime Trends for Selected Cities",
            labels={'Opfer_insgesamt': 'Total Crimes'}
        )
    else:
        # Show overall trend
        overall_trend = all_years_combined.groupby('Year')['Opfer_insgesamt'].sum().reset_index()
        fig = px.line(
            overall_trend,
            x='Year',
            y='Opfer_insgesamt',
            title="Overall Crime Trend in Germany",
            labels={'Opfer_insgesamt': 'Total Crimes'}
        )
    
    fig.update_layout(template=template, height=400)
    return fig

def create_crime_type_analysis(crime_type, start_year, end_year, template):
    years_in_range = [year for year in range(start_year, end_year + 1) if year in crime_categories_data]
    
    if not years_in_range:
        fig = go.Figure()
        fig.update_layout(title="No crime type data available", template=template, height=400)
        return fig
    
    yearly_totals = []
    for year in years_in_range:
        yearly_data = crime_categories_data[year]
        total_crimes = yearly_data['Opfer_insgesamt'].sum() if 'Opfer_insgesamt' in yearly_data.columns else yearly_data['Total'].sum()
        yearly_totals.append(total_crimes)
    
    fig = px.line(
        x=years_in_range,
        y=yearly_totals,
        title=f"Overall Crime Trend ({start_year}-{end_year})",
        labels={'x': 'Year', 'y': 'Total Crimes'}
    )
    
    fig.update_layout(template=template, height=400)
    return fig

def create_seasonal_patterns(start_year, end_year, template):
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    seasonal_data = []
    for year in range(start_year, end_year + 1):
        base = 100 + (year - start_year) * 10
        monthly_crimes = [
            base * 0.9, base * 0.85, base * 0.95, base * 1.0, base * 1.1, base * 1.2,
            base * 1.3, base * 1.25, base * 1.15, base * 1.05, base * 0.95, base * 0.9
        ]
        
        for i, month in enumerate(months):
            seasonal_data.append({
                'Year': year,
                'Month': month,
                'Crime_Count': monthly_crimes[i] + random.randint(-10, 10)
            })
    
    df_seasonal = pd.DataFrame(seasonal_data)
    
    fig = px.line(
        df_seasonal, 
        x='Month', 
        y='Crime_Count', 
        color='Year',
        title="Seasonal Crime Patterns",
        labels={'Crime_Count': 'Number of Crimes', 'Month': 'Month'}
    )
    
    fig.update_layout(template=template, height=400)
    return fig

def create_crime_type_distribution(start_year, end_year, template):
    crime_type_totals = {}
    
    for year in range(start_year, end_year + 1):
        if year in crime_categories_data:
            year_data = crime_categories_data[year]
            for crime_type in crime_categories:
                if crime_type in year_data.columns:
                    total = year_data[crime_type].sum()
                    crime_type_totals[crime_type] = crime_type_totals.get(crime_type, 0) + total
    
    if not crime_type_totals:
        # Use the shortened crime categories instead of long names
        crime_type_totals = {crime: (i+1)*1000 for i, crime in enumerate(crime_categories[:8])}
    
    # Create shorter labels for the pie chart
    def create_short_label(crime_name, max_length=20):
        """Create shorter labels for pie chart display"""
        if len(crime_name) <= max_length:
            return crime_name
        
        # Common German crime type shortenings
        short_mapping = {
            'Sexualstraftaten': 'Sexualstraftaten',
            'Raub & Erpressung': 'Raub & Erpressung',
            'Raub Banken/Post': 'Raub Banken/Post',
            'Raub Geschäfte': 'Raub Geschäfte',
            'Handtaschenraub': 'Handtaschenraub',
            'Raub auf Straßen': 'Raub auf Straßen',
            'Raub in Wohnungen': 'Raub in Wohnungen',
            'Schwere Körperverletzung': 'Schwere KV',
            'Einfache Körperverletzung': 'Einfache KV',
            'Widerstand gegen Beamte': 'Widerstand Beamte',
            'Angriff auf Beamte': 'Angriff Beamte',
            'Gewaltkriminalität': 'Gewaltkriminalität',
            'Mord & Totschlag': 'Mord & Totschlag'
        }
        
        # Use mapping if available, otherwise truncate
        return short_mapping.get(crime_name, crime_name[:max_length-3] + '...')
    
    # Prepare data for pie chart with shorter labels
    labels = []
    values = []
    full_names = []
    
    for crime_type, total in crime_type_totals.items():
        labels.append(create_short_label(crime_type))
        values.append(total)
        full_names.append(crime_type)
    
    # Create the pie chart
    fig = px.pie(
        names=labels,
        values=values,
        title="Distribution of Crime Types",
        hole=0.4,
        custom_data=[full_names]  # Store full names for hover
    )
    
    # Update hover template to show full names
    fig.update_traces(
        hovertemplate="<b>%{customdata[0]}</b><br>Count: %{value:,}<br>Percentage: %{percent}<extra></extra>",
        textposition='inside',
        textinfo='percent+label',
        insidetextorientation='horizontal'
    )
    
    # Improve layout for better text display
    fig.update_layout(
        template=template,
        height=500,
        margin=dict(t=50, b=50, l=50, r=50),
        showlegend=False,  # We're using labels instead of legend
        annotations=[],  # Remove default annotations
        uniformtext_minsize=10,
        uniformtext_mode='hide'
    )
    
    return fig

def create_crime_type_trends_chart(selected_cities, template):
    """
    Create a chart showing how different crime types changed over 2019-2024 in selected cities
    """
    if not selected_cities:
        fig = go.Figure()
        fig.update_layout(
            title="Please select cities to view crime type trends",
            template=template,
            height=500
        )
        return fig
    
    # Collect data for all years and selected cities
    trend_data = []
    
    for year in years:
        if year in crime_categories_data:
            year_data = crime_categories_data[year]
            city_data = year_data[year_data['Stadt'].isin(selected_cities)]
            
            for city in selected_cities:
                city_year_data = city_data[city_data['Stadt'] == city]
                if not city_year_data.empty:
                    for crime_type in crime_categories:
                        if crime_type in city_year_data.columns:
                            crime_count = city_year_data[crime_type].sum()
                            trend_data.append({
                                'Year': year,
                                'City': city.title(),
                                'Crime_Type': crime_type,
                                'Count': crime_count
                            })
    
    if not trend_data:
        # Fallback: create sample data
        for year in years:
            for city in selected_cities:
                for i, crime_type in enumerate(crime_categories):
                    base_value = 1000 + (2024 - year) * 100 + i * 200
                    random_variation = random.randint(-50, 50)
                    trend_data.append({
                        'Year': year,
                        'City': city.title(),
                        'Crime_Type': crime_type,
                        'Count': max(0, base_value + random_variation)
                    })
    
    df_trends = pd.DataFrame(trend_data)
    
    # Create subplots - one for each crime type
    crime_types = df_trends['Crime_Type'].unique()
    n_cols = 2
    n_rows = (len(crime_types) + n_cols - 1) // n_cols
    
    fig = make_subplots(
        rows=n_rows, 
        cols=n_cols,
        subplot_titles=[f"{crime_type}" for crime_type in crime_types],
        vertical_spacing=0.1,
        horizontal_spacing=0.08
    )
    
    colors = px.colors.qualitative.Set3
    
    for i, crime_type in enumerate(crime_types):
        row = (i // n_cols) + 1
        col = (i % n_cols) + 1
        
        crime_data = df_trends[df_trends['Crime_Type'] == crime_type]
        
        for j, city in enumerate(selected_cities):
            city_data = crime_data[crime_data['City'] == city.title()]
            if not city_data.empty:
                fig.add_trace(
                    go.Scatter(
                        x=city_data['Year'],
                        y=city_data['Count'],
                        name=f"{city.title()} - {crime_type}",
                        mode='lines+markers',
                        line=dict(width=2),
                        marker=dict(size=6),
                        legendgroup=city.title(),
                        showlegend=(i == 0),  # Only show legend for first subplot
                        line_color=colors[j % len(colors)]
                    ),
                    row=row,
                    col=col
                )
    
    fig.update_layout(
        title_text="Crime Type Trends by City (2019-2024)",
        template=template,
        height=600,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update y-axis titles
    for i in range(1, n_rows * n_cols + 1):
        fig.update_yaxes(title_text="Count", row=(i-1)//n_cols + 1, col=(i-1)%n_cols + 1)
    
    return fig

def create_crime_density_correlation(start_year, end_year, template):
    correlation_data = all_years_combined[
        (all_years_combined['Year'] >= start_year) & 
        (all_years_combined['Year'] <= end_year)
    ]
    
    city_stats = correlation_data.groupby('Stadt')['Opfer_insgesamt'].sum().reset_index()
    city_stats['Population_Density'] = [random.randint(500, 5000) for _ in range(len(city_stats))]
    
    fig = px.scatter(
        city_stats,
        x='Population_Density',
        y='Opfer_insgesamt',
        size='Opfer_insgesamt',
        color='Opfer_insgesamt',
        hover_name='Stadt',
        title="Crime vs Population Density Analysis",
        labels={'Opfer_insgesamt': 'Total Crimes', 'Population_Density': 'Population Density'}
    )
    
    fig.update_layout(template=template, height=450)
    return fig

def create_crime_forecast(template):
    yearly_totals = all_years_combined.groupby('Year')['Opfer_insgesamt'].sum().reset_index()
    
    if len(yearly_totals) < 2:
        fig = go.Figure()
        fig.update_layout(title="Not enough data for forecasting", template=template, height=400)
        return fig
    
    last_year = yearly_totals['Year'].max()
    forecast_year = last_year + 1
    recent_trend = yearly_totals['Opfer_insgesamt'].iloc[-1] - yearly_totals['Opfer_insgesamt'].iloc[-2]
    forecast_value = max(0, yearly_totals['Opfer_insgesamt'].iloc[-1] + recent_trend)
    
    forecast_data = yearly_totals.copy()
    forecast_data['Type'] = 'Historical'
    
    forecast_row = pd.DataFrame({
        'Year': [forecast_year],
        'Opfer_insgesamt': [forecast_value],
        'Type': ['Forecast']
    })
    
    forecast_data = pd.concat([forecast_data, forecast_row])
    
    fig = px.line(
        forecast_data[forecast_data['Type'] == 'Historical'],
        x='Year',
        y='Opfer_insgesamt',
        title=f"Crime Trend & Forecast (Projected {forecast_year})",
        labels={'Opfer_insgesamt': 'Total Crimes'}
    )
    
    fig.add_scatter(
        x=[forecast_year],
        y=[forecast_value],
        mode='markers+lines',
        line=dict(dash='dash', color='red'),
        marker=dict(size=12, color='red'),
        name='Forecast'
    )
    
    fig.update_layout(template=template, height=400)
    return fig

def create_data_table(data):
    if data.empty:
        return [], []
    
    table_data = data.groupby(['Stadt', 'Year'])['Opfer_insgesamt'].sum().reset_index()
    table_data = table_data.sort_values(['Year', 'Opfer_insgesamt'], ascending=[True, False])
    
    columns = [
        {"name": "City", "id": "Stadt"},
        {"name": "Year", "id": "Year"},
        {"name": "Total Crimes", "id": "Opfer_insgesamt", "type": "numeric", "format": {"specifier": ","}}
    ]
    
    return columns, table_data.to_dict('records')

print("Chart creation functions defined")

Chart creation functions defined


# 9.5

In [10]:
# ====== CRIME TYPE TRENDS FUNCTION ======
def create_crime_type_trends_chart(selected_cities, template):
    """
    Create a chart showing how different crime types changed over 2019-2024 in selected cities
    """
    if not selected_cities:
        fig = go.Figure()
        fig.update_layout(
            title="Please select cities to view crime type trends",
            template=template,
            height=500
        )
        return fig
    
    # Collect data for all years and selected cities
    trend_data = []
    
    for year in years:
        if year in crime_categories_data:
            year_data = crime_categories_data[year]
            
            # Filter for selected cities
            city_data = year_data[year_data['Stadt'].isin(selected_cities)]
            
            # Use short crime names if available, otherwise use original
            crime_column = 'Straftat_Short' if 'Straftat_Short' in city_data.columns else 'Straftat'
            
            if crime_column not in city_data.columns:
                print(f"No crime data column in {year} data")
                continue
                
            for city in selected_cities:
                city_year_data = city_data[city_data['Stadt'] == city]
                if not city_year_data.empty:
                    # Group by crime type and sum the Total column
                    crime_totals = city_year_data.groupby(crime_column)['Total'].sum().reset_index()
                    
                    for _, row in crime_totals.iterrows():
                        trend_data.append({
                            'Year': year,
                            'City': city.title(),
                            'Crime_Type': row[crime_column],
                            'Count': row['Total']
                        })
    
    if not trend_data:
        # Fallback: create sample data based on actual crime types
        print("Using fallback data for crime type trends")
        for year in years:
            for city in selected_cities:
                for i, crime_type in enumerate(crime_categories[:8]):
                    base_value = 1000 + (2024 - year) * 100 + i * 200
                    random_variation = random.randint(-50, 50)
                    trend_data.append({
                        'Year': year,
                        'City': city.title(),
                        'Crime_Type': crime_type,
                        'Count': max(0, base_value + random_variation)
                    })
    
    df_trends = pd.DataFrame(trend_data)
    
    # Get top crime types by total count across all years
    top_crime_types = df_trends.groupby('Crime_Type')['Count'].sum().nlargest(8).index.tolist()
    
    # Ensure sexual crimes are included if they exist in data
    sexual_crime_keywords = ['sexualstraftaten', 'vergewaltigung', 'sexuelle', 'nötigung', 'übergriff']
    
    # Check if we have any sexual crimes in the data
    all_crime_types = df_trends['Crime_Type'].unique()
    sexual_crimes_in_data = [crime for crime in all_crime_types 
                           if any(keyword in str(crime).lower() for keyword in sexual_crime_keywords)]
    
    # If we have sexual crimes but they're not in top 8, replace the least common one
    if sexual_crimes_in_data and sexual_crimes_in_data[0] not in top_crime_types and len(top_crime_types) >= 8:
        # Replace the least common crime type with the sexual crime
        least_common = top_crime_types[-1]
        top_crime_types[-1] = sexual_crimes_in_data[0]
        print(f"Prioritized sexual crime '{sexual_crimes_in_data[0]}' over '{least_common}'")
    
    if not top_crime_types:
        top_crime_types = crime_categories[:8]
    
    # Filter data to only include top crime types
    df_filtered = df_trends[df_trends['Crime_Type'].isin(top_crime_types)]
    
    # Create subplots - one for each crime type
    n_cols = 2
    n_rows = (len(top_crime_types) + n_cols - 1) // n_cols
    
    fig = make_subplots(
        rows=n_rows, 
        cols=n_cols,
        subplot_titles=[f"{crime_type}" for crime_type in top_crime_types],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    colors = px.colors.qualitative.Set3
    
    for i, crime_type in enumerate(top_crime_types):
        row = (i // n_cols) + 1
        col = (i % n_cols) + 1
        
        crime_data = df_filtered[df_filtered['Crime_Type'] == crime_type]
        
        for j, city in enumerate(selected_cities):
            city_data = crime_data[crime_data['City'] == city.title()]
            if not city_data.empty:
                fig.add_trace(
                    go.Scatter(
                        x=city_data['Year'],
                        y=city_data['Count'],
                        name=f"{city.title()}",
                        mode='lines+markers',
                        line=dict(width=3),
                        marker=dict(size=8),
                        legendgroup=city.title(),
                        showlegend=(i == 0),  # Only show legend for first subplot
                        line_color=colors[j % len(colors)]
                    ),
                    row=row,
                    col=col
                )
    
    fig.update_layout(
        title_text="Crime Type Trends by City (2019-2024) - Including Sexual Crimes",
        template=template,
        height=800 if n_rows > 2 else 600,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update axis titles
    for i in range(1, n_rows * n_cols + 1):
        fig.update_yaxes(title_text="Number of Victims", row=(i-1)//n_cols + 1, col=(i-1)%n_cols + 1)
        fig.update_xaxes(title_text="Year", row=(i-1)//n_cols + 1, col=(i-1)%n_cols + 1)
    
    return fig

print("Crime type trends function defined")

Crime type trends function defined


# 10

In [11]:
# ====== MAIN CALLBACK ======
@app.callback(
    [Output("crime-map", "figure"),
     Output("top-cities-chart", "figure"),
     Output("crime-trends-chart", "figure"),
     Output("crime-type-analysis", "figure"),
     Output("seasonal-patterns-chart", "figure"),
     Output("crime-type-distribution", "figure"),
     Output("crime-density-correlation", "figure"),
     Output("crime-forecast", "figure"),
     Output("crime-type-trends-chart", "figure"),  # Back to showing all crime types
     Output("crime-data-table", "columns"),
     Output("crime-data-table", "data")],
    [Input("year-slider", "value"),
     Input("crime-type-dropdown", "value"),
     Input("top-cities-dropdown", "value"),
     Input("crime-trend-cities-dropdown", "value"),
     Input("theme-store", "data")]
)
def update_all_charts(year_range, crime_type, selected_cities, crime_trend_cities, theme):
    start_year, end_year = year_range
    template = "plotly_dark" if theme == "dark" else "plotly_white"
    
    # Filter data based on year range
    filtered_data = all_years_combined[
        (all_years_combined['Year'] >= start_year) & 
        (all_years_combined['Year'] <= end_year)
    ]
    
    # 1. Crime Map
    map_fig = create_crime_map(filtered_data, template, year_range)
    
    # 2. Top Cities Chart
    top_cities_fig = create_top_cities_chart(filtered_data, template)
    
    # 3. Crime Trends Chart
    trends_fig = create_trends_chart(start_year, end_year, selected_cities, template)
    
    # 4. Crime Type Analysis
    crime_type_fig = create_crime_type_analysis(crime_type, start_year, end_year, template)
    
    # 5. Seasonal Patterns
    seasonal_fig = create_seasonal_patterns(start_year, end_year, template)
    
    # 6. Crime Type Distribution
    distribution_fig = create_crime_type_distribution(start_year, end_year, template)
    
    # 7. Crime Density Correlation
    correlation_fig = create_crime_density_correlation(start_year, end_year, template)
    
    # 8. Crime Forecast
    forecast_fig = create_crime_forecast(template)
    
    # 9. BACK TO ORIGINAL: Crime Type Trends Chart (all types including sexual crimes)
    crime_type_trends_fig = create_crime_type_trends_chart(crime_trend_cities, template)
    
    # 10. Data Table
    table_columns, table_data = create_data_table(filtered_data)
    
    return (map_fig, top_cities_fig, trends_fig, crime_type_fig, 
            seasonal_fig, distribution_fig, correlation_fig, forecast_fig,
            crime_type_trends_fig,  # BACK TO ORIGINAL
            table_columns, table_data)

print("Main callback defined")

Main callback defined


# 11

In [ ]:
# ====== RUN APP ======
print("the app is running")
print("================================")
if __name__ == "__main__":
    app.run(debug=True, port=2124)

#i have added the baraces at end

the app is running


Map data for [2019, 2024] - Cities with crime data: 137
Map data for [2019, 2024] - Total areas: 403


/var/folders/7y/st4m4q0j5r7d26l_006982gw0000gn/T/ipykernel_4052/3939038048.py:27: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



Map data for [2019, 2024] - Cities with crime data: 137
Map data for [2019, 2024] - Total areas: 403
Map data for [2019, 2024] - Cities with crime data: 137
Map data for [2019, 2024] - Total areas: 403


/var/folders/7y/st4m4q0j5r7d26l_006982gw0000gn/T/ipykernel_4052/3939038048.py:45: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


